In [1]:
!pip install keras_ocr

   ---------------------------------------- 9.5/9.5 MB 134.7 kB/s eta 0:00:00
   -------------------------------------- 948.0/948.0 kB 141.6 kB/s eta 0:00:00
   ---------------------------------------- 1.4/1.4 MB 176.5 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import matplotlib.pyplot as plt
import keras_ocr
import cv2
import math
import numpy as np
import os
import pandas as pd

def midpoint(x1, y1, x2, y2):
    return int((x1 + x2) / 2), int((y1 + y2) / 2)

pipeline = keras_ocr.pipeline.Pipeline()

def inpaint_text(img_path, pipeline):
    """Removes detected text from an image and extracts the text."""
    img = keras_ocr.tools.read(img_path)  # Read the image
    prediction_groups = pipeline.recognize([img])  # Detect text
    mask = np.zeros(img.shape[:2], dtype="uint8")
    extracted_texts = []  # Store extracted text
    
    for box in prediction_groups[0]:
        text = box[0]  # Extract text
        extracted_texts.append(text)  # Store it

        x0, y0 = box[1][0]
        x1, y1 = box[1][1]
        x2, y2 = box[1][2]
        x3, y3 = box[1][3]
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mid1 = midpoint(x0, y0, x3, y3)
        thickness = int(math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2))
        
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mid1), 255, thickness)
        img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
    
    return img, extracted_texts

# Directory paths
directory = "keypoints_only_new"
output_directory = "anonymized_data"
csv_output_path = "anonymized_data.csv"

# Initialize data storage
text_data = []
i = 1237

for filename in os.listdir(directory):
    if filename.endswith(".jpg"):
        image_path = os.path.join(directory, filename)
        filename_without_extension = os.path.splitext(filename)[0]
        
        # Apply inpaint function
        inpainted_image, extracted_texts = inpaint_text(image_path, pipeline)
        
        # Save inpainted image
        output_path = os.path.join(output_directory, f"{i}.jpg")
        cv2.imwrite(output_path, inpainted_image)
        
        # Save extracted text
        for text in extracted_texts:
            text_data.append([filename, text])
        
        i += 1

# Convert extracted text data into CSV
text_df = pd.DataFrame(text_data, columns=["Image Name", "Extracted Text"])
text_df.to_csv(csv_output_path, index=False)

print(f"Text extraction complete. CSV saved at {csv_output_path}")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\ajani\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\ajani\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 985, in launch_instance
    app.start()
  File "C:\Users\ajani\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 712, in start
    

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [1]:
import os
import shutil

# Source and target directories
source_dir = 'anonymized_data'
target_dir = 'anonymized_renamed_data'

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# List all files in the source directory and sort them
files = sorted([f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))])

# Rename and copy files
for i, filename in enumerate(files, start=1):
    ext = os.path.splitext(filename)[1]  # Keep original extension
    new_name = f"IMG{i:04d}{ext}"
    src_path = os.path.join(source_dir, filename)
    dst_path = os.path.join(target_dir, new_name)
    shutil.copy2(src_path, dst_path)

print(f"Renamed {len(files)} images and saved to '{target_dir}'")


Renamed 1198 images and saved to 'anonymized_renamed_data'
